# Notes
* Tratar attribute.confidentiality.data.amount.* de forma separada, pois são valores, e não one-hot encodings.
* attribute.confidentiality.data.variety.Payment também.

In [13]:
import pandas as pd
from functools import reduce

pd.set_option('display.max_columns', None)

In [22]:
def coalesce(df, col_names):
    cols = [df[col] for col in col_names]
    return reduce(lambda x, y: x.fillna(y), cols)

In [2]:
def get_vcdb() -> pd.DataFrame:
    columns = open("header.txt", "r").read().splitlines()
    return pd.read_csv(
            "data/vcdb.csv",
            low_memory=False,
            usecols=columns
    )

In [3]:
df = get_vcdb()

# Manipular variáveis one-hot encoded

# Incident

In [73]:
# Replace 'your_file.csv' with your actual CSV file path
input_csv = "./data/vcdb.csv"
output_csv = "transformed_file.csv"

# Process the CSV
transformed_data = process_csv(input_csv, output_csv)

if transformed_data is not None:
    # Preview the results
    #preview_transformation(transformed_data)
    
    # Show summary statistics
    print(f"\nSummary:")
    print(f"Total rows: {len(transformed_data)}")
    print(f"Total columns: {len(transformed_data.columns)}")
    
    # Show non-null counts for new categorical columns
    categorical_cols = [col for col in transformed_data.columns 
                        if len(col.split('.')) == 2 and col != 'incident_id']
    if categorical_cols:
        print(f"\nNon-null counts for categorical columns:")
        for col in categorical_cols:
            non_null_count = transformed_data[col].count()
            print(f"  {col}: {non_null_count}")

Reading CSV file: ./data/vcdb.csv
Original shape: (10394, 419)
Original columns: 419
Transforming one-hot encoded columns...
Transformed shape: (10394, 27)
Transformed columns: 27
Transformed data saved to: transformed_file.csv

Summary:
Total rows: 10394
Total columns: 27

Non-null counts for categorical columns:
  action.variety: 665
  action.vector: 665
  action.result: 134
  action.target: 665
  actor.motive: 5843
  attribute.duration: 20
  attribute.variety: 2779
  attribute.data_disclosure: 10394
  attribute.data_victim: 7327
  attribute.state: 7535
  discovery_method.variety: 45


In [66]:
cve_columns = [c for c in transformed_data.columns if c.split(".")[-1] == 'cve']
notes_columns = [c for c in transformed_data.columns if c.split(".")[-1] == 'notes']
name_columns = [c for c in transformed_data.columns if c.split(".")[-1] == 'name']

In [67]:
one_hot_processed = transformed_data.copy()

In [68]:
one_hot_processed["cve"] = coalesce(one_hot_processed, cve_columns)
one_hot_processed["notes"] = coalesce(one_hot_processed, notes_columns)
one_hot_processed["threat_name"] = coalesce(one_hot_processed, name_columns)

one_hot_processed.drop(columns=cve_columns + notes_columns + name_columns, inplace=True)

In [74]:
one_hot_processed.head()

,attribute.confidentiality.data_total,discovery_notes,incident_id,action.variety,action.vector,action.result,action.target,actor.motive,attribute.duration,attribute.variety,attribute.data_disclosure,attribute.data_victim,attribute.state,discovery_method.variety,cve,notes,threat_name
0,1.0,NaN,0001AA7F-C601-424A-B2B8-BE6C9F5164E7,None,None,None,None,None,None,None,Yes,None,None,None,NaN,NaN,NaN
1,NaN,NaN,0eb22d90-8e87-11ec-a600-657ec2694f8f,None,None,None,None,None,None,None,Yes,Customer,Stored,None,NaN,NaN,NaN
2,NaN,NaN,0008DADB-E83D-4278-A19A-CEE01610CF43,None,None,None,None,Financial,None,Loss,Potentially,Patient,Stored unencrypted,None,NaN,NaN,NaN
3,NaN,NaN,000D403E-2DC9-4EA7-9294-BD3938D1C3C7,None,None,None,None,None,None,Loss,Potentially,Patient,Stored unencrypted,None,NaN,NaN,NaN
4,NaN,NaN,0012CC25-9167-40D8-8FE3-3D0DFD8FB6BB,None,None,None,None,Financial,None,None,Yes,Customer,Stored unencrypted,None,NaN,NaN,NaN


In [ ]:
def transform_country_columns(df, pattern=None):
    """
    Transform one-hot encoded country columns to categorical format.
    
    Args:
        df: pandas DataFrame with one-hot encoded columns
        pattern: string pattern to match columns (e.g., 'actor.partner.country.')
                If None, will auto-detect country patterns
    
    Returns:
        pandas DataFrame with transformed columns
    """
    result_df = df.copy()
    
    # Auto-detect country column patterns if not provided
    if pattern is None:
        # Find all columns that end with 2-letter country codes or 'Unknown'
        country_patterns = set()
        for col in df.columns:
            parts = col.split('.')
            if len(parts) >= 4 and 'country' in col:
                # Extract pattern up to 'country'
                country_idx = None
                for i, part in enumerate(parts):
                    if part == 'country':
                        country_idx = i
                        break
                if country_idx is not None:
                    pattern_base = '.'.join(parts[:country_idx+1]) + '.'
                    country_patterns.add(pattern_base)
        
        patterns = list(country_patterns)
    else:
        patterns = [pattern]
    
    columns_to_drop = []
    
    for pattern in patterns:
        # Find all columns matching this pattern
        matching_cols = [col for col in df.columns if col.startswith(pattern)]
        
        if not matching_cols:
            continue
        
        # Extract the base parts (everything before 'country')
        parts = pattern.rstrip('.').split('.')
        country_idx = None
        for i, part in enumerate(parts):
            if part == 'country':
                country_idx = i
                break
        
        if country_idx is None:
            continue
        
        # Create new column names
        # For 'actor.partner.country.' -> 'actor' and 'country'
        first_part = parts[0]  # 'actor'
        third_part = 'country'  # always 'country'
        
        new_col_name_1 = first_part  # 'actor'
        new_col_name_2 = third_part  # 'country'
        
        # Create value series for coalescing
        value_series_1 = []  # For the first column (actor type)
        value_series_2 = []  # For the second column (country code)
        
        for col in matching_cols:
            # Extract the country code (last part)
            country_code = col.split('.')[-1]
            
            # Extract the middle part (partner, external, etc.)
            middle_part = parts[1] if len(parts) > 1 else 'unknown'
            
            # Create series for first column (actor type)
            series_1 = pd.Series(dtype='object', index=df.index)
            mask = (df[col] == 1) | (df[col] == True)
            series_1[mask] = middle_part
            value_series_1.append(series_1)
            
            # Create series for second column (country)
            series_2 = pd.Series(dtype='object', index=df.index)
            series_2[mask] = country_code
            value_series_2.append(series_2)
        
        # Coalesce to get final values
        if value_series_1:
            result_df[new_col_name_1] = coalesce(value_series_1)
        if value_series_2:
            result_df[new_col_name_2] = coalesce(value_series_2)
        
        # Mark columns for removal
        columns_to_drop.extend(matching_cols)
    
    # Drop original one-hot columns
    result_df = result_df.drop(columns=columns_to_drop)
    
    return result_df

def transform_specific_country_pattern(df, pattern):
    """
    Transform specific country pattern with more control.
    
    Args:
        df: pandas DataFrame
        pattern: exact pattern like 'actor.partner.country.'
    
    Returns:
        pandas DataFrame with new columns
    """
    result_df = df.copy()
    
    # Find matching columns
    matching_cols = [col for col in df.columns if col.startswith(pattern)]
    
    if not matching_cols:
        print(f"No columns found matching pattern: {pattern}")
        return result_df
    
    # Parse the pattern to get column names
    parts = pattern.rstrip('.').split('.')
    
    if len(parts) < 3:
        print(f"Pattern should have at least 3 parts: {pattern}")
        return result_df
    
    # Create new column names: first part and third part
    new_col_1 = parts[0]  # e.g., 'actor'
    new_col_2 = parts[2]  # e.g., 'country'
    
    # Get the middle part value (e.g., 'partner')
    middle_value = parts[1]
    
    # Create series for coalescing
    value_series_1 = []  # For actor type
    value_series_2 = []  # For country code
    
    for col in matching_cols:
        # Extract country code (last part)
        country_code = col.split('.')[-1]
        
        # Create series where 1/True becomes the respective value
        series_1 = pd.Series(dtype='object', index=df.index)
        series_2 = pd.Series(dtype='object', index=df.index)
        
        mask = (df[col] == 1) | (df[col] == True)
        series_1[mask] = middle_value  # 'partner'
        series_2[mask] = country_code  # 'US', 'UK', etc.
        
        value_series_1.append(series_1)
        value_series_2.append(series_2)
    
    # Coalesce to get final columns
    result_df[new_col_1] = coalesce(value_series_1)
    result_df[new_col_2] = coalesce(value_series_2)
    
    # Drop original columns
    result_df = result_df.drop(columns=matching_cols)
    
    return result_df